In [1]:
import os

from pathlib import Path

import pandas as pd

from star_parser import StarParser, StarTabDf

In [6]:
data_folder = Path(r'/mnt/DATA/andrea/2021_JBP3_reprocess/cryolo_pick_random50_general')
starfiles = data_folder.glob('*doseweighted.star')
collated = []
for star in starfiles:
    data = StarParser(star).parse()['data_']
    mic_name = 'Micrographs/' + star.name.replace('star','mrc')
    data.fill_column('MicrographName', mic_name, create=True, store=True)
    new_order = [data.columns()[-1]] + data.columns()[:-1]
    data.reorder_columns(new_order, store=True)
    data.remove_columns(['ClassNumber',	'AnglePsi',	'AutopickFigureOfMerit'], store=True)
    collated.append(data.to_df())


In [8]:
merged = pd.concat(collated)
collated_star = StarTabDf(merged)

In [10]:
with open(data_folder/'cryolo_pick_random50_general.star', 'w') as f:
    f.write(collated_star.to_star())